In [1]:
import gpytorch
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torchvision import transforms

gpytorch.functions.use_toeplitz = False

In [2]:
train_mnist = torchvision.datasets.ImageFolder('/scratch/bw462/flattened/', transform=transforms.Compose([
                        transforms.Scale((28,28)),
                        transforms.ToTensor()
                   ]))                                              
"""
test_mnist = torchvision.datasets.ImageFolder('/tmp', split='test',
                                        download=True, transform=transforms.Compose([
                       transforms.ToTensor()
                   ]))
"""

"\ntest_mnist = torchvision.datasets.ImageFolder('/tmp', split='test',\n                                        download=True, transform=transforms.Compose([\n                       transforms.ToTensor()\n                   ]))\n"

In [3]:
class FeatureExtractor(nn.Sequential):
    
    def __init__(self):
        super(FeatureExtractor, self).__init__(nn.Conv2d(1, 32, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2),
                                 nn.Conv2d(32, 64, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2))
        
class Bottleneck(nn.Sequential):
    
    def __init__(self):
        super(Bottleneck, self).__init__(nn.Linear(64*7*7, 128),
                                         nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128, 128),
                                 nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128,64),
                                 nn.BatchNorm1d(64))

class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.bottleneck = Bottleneck()
        self.final_layer = nn.Sequential(
                                 nn.ReLU(),
                                 nn.Linear(64,1319))
    
    def forward(self, x):
        input_x = x[:,0,:,:].unsqueeze(1)
        features = self.feature_extractor(input_x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        classification = self.final_layer(bottlenecked_features)
        return classification
        

In [4]:
train_data_loader = torch.utils.data.DataLoader(train_mnist, shuffle=True, pin_memory=True, batch_size=256)

In [5]:
criterion = nn.CrossEntropyLoss()#.cuda()

In [ ]:
model = LeNet()#.cuda() 
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

In [ ]:
model.train()
num_epochs = 20
for i in range(num_epochs):
    for x, y in train_data_loader:
        optimizer.zero_grad()
        x = Variable(x)#.cuda()
        y = Variable(y)#.cuda()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print("Loss: %.3f" % loss.data[0])
    


Loss: 7.100
Loss: 7.008
Loss: 4.493
Loss: 4.069
Loss: 3.454
Loss: 2.978
Loss: 2.567


In [ ]:
model.eval()
test_data_loader = train_data_loader#torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)
avg = 0.
i = 0.
for test_batch_x, test_batch_y in test_data_loader:
    predictions = model(Variable(test_batch_x)).max(-1)[1]#.cuda()).max(-1)[1]
    test_batch_y = Variable(test_batch_y)#.cuda()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.
    print('Accuracy: %.4f' % (avg / i))

In [ ]:
list(model.bottleneck.modules())[-1].weight.data.fill_(1)
None

In [ ]:
from gpytorch.kernels import RBFKernel, GridInterpolationKernel

class DeepKernel(gpytorch.Module):
    def __init__(self, model):
        super(DeepKernel, self).__init__()
        self.feature_extractor = model.feature_extractor
        self.bottleneck = model.bottleneck
        self.gp_layer = GPLayer()
        
    def forward(self, x):
        features = self.feature_extractor(x[:,0,:,:].unsqueeze(1))
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        gp_output = self.gp_layer(0.1*bottlenecked_features)
        return gp_output
    
    
class LatentFunction(gpytorch.AdditiveGridInducingPointModule):
    def __init__(self):
        super(LatentFunction, self).__init__(grid_size=256, grid_bounds=[(-7, 7)],
                                             n_components=64, mixing_params=False, sum_output=False)
        cov_module = RBFKernel()
        cov_module.initialize(log_lengthscale=2)
        self.cov_module = cov_module
        
    def forward(self, x):
        mean = Variable(x.data.new(len(x)).zero_())
        covar = self.cov_module(x)
        return gpytorch.random_variables.GaussianRandomVariable(mean, covar)

    
class GPLayer(gpytorch.GPModel):
    def __init__(self, n_dims=64):
        super(GPLayer, self).__init__(gpytorch.likelihoods.SoftmaxLikelihood(n_features=64, n_classes=1319))
        self.latent_function = LatentFunction()
    
    def forward(self, x):
        res = self.latent_function(x)
        return res
    

In [ ]:
len(train_mnist)

In [ ]:
deep_kernel = DeepKernel(model).cuda()
gp_data_loader = torch.utils.data.DataLoader(train_mnist, batch_size=2048., pin_memory=True, shuffle=True)

In [ ]:
# Find optimal model hyperparameters
deep_kernel.train()
optimizer = torch.optim.Adam(deep_kernel.gp_layer.parameters(), lr=0.01)
optimizer.n_iter = 0
num_epochs = 10
for i in range(num_epochs):
    for j, (train_x_batch, train_y_batch) in enumerate(gp_data_loader):
        train_x_batch = Variable(train_x_batch).cuda()
        train_y_batch = Variable(train_y_batch).cuda()
        optimizer.zero_grad()
        output = deep_kernel(train_x_batch)
        loss = -deep_kernel.gp_layer.marginal_log_likelihood(output, train_y_batch, n_data=len(train_mnist))
        loss.backward()
        optimizer.n_iter += 1
        print('Iter %d/%d - Loss: %.3f' % (
            i + 1, num_epochs, loss.data[0],
        ))
        optimizer.step()
        
    deep_kernel.eval()
    test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

    avg = 0.
    i = 0.
    for test_batch_x, test_batch_y in test_data_loader:
        predictions = deep_kernel(Variable(test_batch_x).cuda()).argmax()
        test_batch_y = Variable(test_batch_y).cuda()
        avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
        i += 1.

    print('Score')
    print(avg / i)
    deep_kernel.train()


In [ ]:
deep_kernel.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

for test_batch_x, test_batch_y in test_data_loader:
    predictions = deep_kernel(Variable(test_batch_x).cuda()).representation().max(-1)[1]
    test_batch_y = Variable(test_batch_y).cuda()
    avg += torch.eq(predictions, test_batch_y).float().mean().data[0]
    i += 1.
print('Accuracy: %.4f' % (avg / i))
